In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import csv

In [4]:
path1 = r"C:\Users\ncwbr\Desktop\hw3-mnist-data.csv"
path2 = r"C:\Users\ncwbr\Desktop\hw3-mnist-query.csv"
data = np.loadtxt(path1, delimiter=",")
query = np.loadtxt(path2, delimiter=",")

In [5]:
def brute_force(data, query, r):
    neighbors = []

    for x in query:
        d = np.sqrt(np.sum((x - data) ** 2, axis = 1))
        index = np.where(d <= r)[0]
        neighbors.append(index.tolist())
    return neighbors

r = 8
start = time.time()
bf_neighbors = brute_force(data, query, r)
end = time.time()
t_bf = end - start

print(f"t_bf = {t_bf:.4f} seconds")

t_bf = 50.3418 seconds


In [18]:
with open("bf-out.csv", "w", newline="") as f:
    writer = csv.writer(f, delimiter=" ")
    writer.writerows(bf_neighbors)

max_cols = max(len(r) for r in bf_neighbors)
rows = [ row + [""]*(max_cols - len(row)) for row in bf_neighbors]
df = pd.DataFrame(rows)
df.to_excel("bf-out.xlsx", index=False, header=False)

In [7]:
L, K, r = 200, 20, 8
n, d = data.shape
w = np.random.randn(L, K, d)
b = np.random.rand(L, K) * r

def lsh_bins(data, query, L, K, r):
    bins = [{} for _ in range(L)]

    for i in range(n):
        x = data[i]
        for l in range(L):
            code = np.floor((w[l] @ x + b[l]) / r).astype(int)
            key = tuple(code.tolist())
            if key not in bins[l]:
                bins[l][key] = []
            bins[l][key].append(i)
            
    return bins

bins = lsh_bins(data, query, L, K, r)

In [12]:
def lsh(bins, data, query, L, K, r):
    neighbors = []
    
    for x in query:
        cand = set()
        for l in range(L):
            code = np.floor((w[l] @ x + b[l]) / r).astype(int)
            key = tuple(code.tolist())
            bucket = bins[l].get(key)
            if bucket:
                for y in bucket:
                    cand.add(y)
        neighbors.append(sorted(cand))
            
    return neighbors

start = time.time()
lsh_neighbors = lsh(bins, data, query, L, K, r)
end = time.time()
t_lsh = end - start

print(f"t_lsh = {t_lsh:.4f} seconds")

t_lsh = 2.4182 seconds


In [19]:
with open("lsh-out.csv", "w", newline="") as f:
    writer = csv.writer(f)   # default delimiter=","
    writer.writerows(lsh_neighbors)

max_cols = max(len(r) for r in lsh_neighbors)
rows = [ row + [""]*(max_cols - len(row)) for row in lsh_neighbors]
df = pd.DataFrame(rows)
df.to_excel("lsh-out.xlsx", index=False, header=False)

In [16]:
p1_list = []
p2_list = []

for bf_n, lsh_n in zip(bf_neighbors, lsh_neighbors):
    neigh = set(bf_n)
    neigh_p = set(lsh_n)

    if len(neigh) > 0:
        p1_list.append(len(neigh & neigh_p) / len(neigh))
    else:
        p1_list.append(0.0)

    d = n - len(neigh)
    if d > 0:
        p2_list.append( len(neigh_p - neigh) / d)
    else:
        p2_list.append(0.0)

p1 = sum(p1_list) / len(p1_list)
p2 = sum(p2_list) / len(p2_list)
print(p1)
print(p2)

0.013822417574015443
1.0775862068965517e-07
